In [12]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np
import pickle

In [2]:
market = Market()
competition_db = ADatabase("competition")

In [3]:
competition_db.connect()
data = competition_db.retrieve("data")
competition_db.disconnect()

In [4]:
market.connect()
sp5 = market.retrieve("sp500")
corrs = market.retrieve("price_corr")
market.disconnect()

In [5]:
start_year = datetime.now().year
end_year = start_year + 1

In [16]:
competition_db.connect()
all_models = competition_db.retrieve("models")
for training_year in [4]:
    training_corrs = corrs[corrs["training_year"]==training_year]
    for year in tqdm(range(start_year,end_year)):
        for ticker in tqdm(sp5["Symbol"]):
            try:
                if ticker in data.columns:
                    ticker_data = data.copy()
                    example = corrs[(corrs["ticker"]==ticker) & (corrs["year"]==2022) & (corrs["training_year"]==4)].tail(1).fillna(0)[[x for x in corrs.columns if x not in ["ticker","year","training_year"]]].melt()
                    example["abs_corr"] = [abs(x) for x in example["value"]]
                    factors = list(example.sort_values("abs_corr",ascending=False).head(25)["variable"])
                    factors = [x for x in factors if x in ticker_data.columns]
                    prediction_data = ticker_data[ticker_data["year"]==year]
                    models = all_models[(all_models["ticker"]==ticker) & (all_models["training_year"]==training_year) & (all_models["year"]==year)]
                    models["model"] = [pickle.loads(x) for x in models["model"]]
                    for row in models.iterrows():
                        model = row[1]["model"]
                        api = row[1]["api"]
                        score = row[1]["score"]
                        prediction_data[f"{api}_prediction"] = model.predict(prediction_data[factors])
                        prediction_data[f"{api}_score"] = score
                    prediction_data["ticker"] = ticker
                    included_columns = ["year","week","ticker","training_year"]
                    included_columns.extend([x for x in prediction_data.columns if "score" in x or "prediction" in x])
                    prediction_data["training_year"] = training_year
                    prediction_data["week"] = prediction_data["week"] + 1
                    competition_db.store("recs",prediction_data[included_columns].tail(1))
                else:
                    continue
            except Exception as e:
                print(year,ticker,str(e))
competition_db.disconnect()

 75%|███████████████████████████████████████████████████████████████████████████████████████████▉                              | 380/504 [00:12<00:04, 30.77it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:16<00:00, 16.13s/it]
